In [1]:
import numpy as np
import pandas as pd
import cypari2

from tqdm.notebook import tqdm

In [3]:
pari = cypari2.Pari()
pari.allocatemem(2**32)

PARI stack size set to 4294967296 bytes, maximum size set to 4294967296


In [4]:
pari.variable()

[x, y]

# Find max dimension of the coeff vectors

In [5]:
def min_max_degree(list_of_poly, variables=['q', 't']):
    min_max = {}
    for v in variables:
        min_max[v] = {}
        min_max[v]['max'] = -1000
        min_max[v]['min'] = +1000
        
    #for poly in tqdm(list_of_poly):
    for poly in list_of_poly:
        for v in variables:
            # max degree
            degree = pari.poldegree(poly, v)
            if degree > min_max[v]['max']:
                min_max[v]['max'] = degree
                
            # min degree
            # goes from -100 to 10 and stops
            # at the first non zero coeff
            for i in range(-100, 10):
                i_degree = pari.polcoef(pari(poly), i, v)
                if i_degree != 0:
                    if i < min_max[v]['min']:
                        min_max[v]['min'] = i
                    break
    
    return min_max

In [6]:
def min_max_degree_single(poly, variables=['q', 't']):
    min_max = {}
    for v in variables:
        min_max[v] = {}
        min_max[v]['max'] = -1000
        min_max[v]['min'] = +1000

        # max degree
        degree = pari.poldegree(poly, v)
        min_max[v]['max'] = degree

        # min degree
        # goes from -100 to 10 and stops
        # at the first non zero coeff
        for i in range(-100, 10):
            i_degree = pari.polcoef(pari(poly), i, v)
            if i_degree != 0:
                min_max[v]['min'] = i
                break
    
    return min_max

# Create vector of coeff

In [7]:
def create_coeff_vector(poly, max_q, min_q, max_t, min_t):
    off_q = -min_q
    off_t = -min_t

    coeff = np.zeros((max_q-min_q+1, max_t-min_t+1), dtype=int)

    for idx_q in range(min_q, max_q+1):
        for idx_t in range(min_t, max_t+1):
            coeff[idx_q+off_q, idx_t+off_t] = pari.polcoef(pari.polcoef(poly, idx_t, 't'), idx_q, 'q')
            
    return coeff.reshape(-1)

def create_colnames(max_q, min_q, max_t, min_t):
    colnames = []
    for idx_q in range(min_q, max_q+1):
        for idx_t in range(min_t, max_t+1):
            colnames.append('q'+str(idx_q)+'_'+'t'+str(idx_t))
    return colnames

# up to 17n mirrors

In [ ]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_data")
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_14n_data")
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_15n_data")
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_16n_data")
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_17n_data")

Loading Khovanov polynomials for knots with up to 13 crossings ... done.
Loading Khovanov polynomials for non-alternating knots with 14 crossings ... done.
Loading Khovanov polynomials for non-alternating knots with 15 crossings ... 

# up to 13

In [9]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_data")

Loading Khovanov polynomials for knots with up to 13 crossings ... done.


In [7]:
poly_vectors = [
'KhPolQ_K3a',
'KhPolQ_K4a',
'KhPolQ_K5a',
'KhPolQ_K6a',
'KhPolQ_K7a',
'KhPolQ_K8a',
'KhPolQ_K8n',
'KhPolQ_K9a', 
'KhPolQ_K9n',
'KhPolQ_K10a',
'KhPolQ_K10n',
'KhPolQ_K11a',
'KhPolQ_K11n',
'KhPolQ_K12a',
'KhPolQ_K12n',
'KhPolQ_K13a',
'KhPolQ_K13n']

In [10]:
poly_vectors = [
'KhPolQ_K3a']

In [11]:
min_max_list = []

for poly_vec in poly_vectors:
    for i in tqdm(range(0, len(pari(poly_vec)), 1000)):
        #print(i, i+1000)
        min_max_list.append(min_max_degree(pari(poly_vec)[i:i+1000]))

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])
size_q = max(abs(max_q), abs(min_q))

max_t = max([v['t']['max'] for v in min_max_list])
min_t = min([v['t']['min'] for v in min_max_list])
size_t = max(abs(max_t), abs(min_t))


print('q', min_q, max_q, size_q)
print('t', min_t, max_t, size_t)

  0%|          | 0/1 [00:00<?, ?it/s]

q -9 -1 9
t -3 0 3


In [12]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q, size_t, -size_t) 
        c_list.append(p_coeff)
        c_list.append(p_coeff.reshape((size_q*2 +1 , size_t*2+1))[::-1, ::-1].reshape(-1))

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

KhPolQ_K3a


0it [00:00, ?it/s]

In [13]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q, size_t, -size_t) )
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

In [16]:
coeff_df

,knot_id,q-9_t-3,q-9_t-2,q-9_t-1,q-9_t0,q-9_t1,q-9_t2,q-9_t3,q-8_t-3,q-8_t-2,...,q8_t3,q9_t-3,q9_t-2,q9_t-1,q9_t0,q9_t1,q9_t2,q9_t3,is_alternating,number_of_crossings
0,KhPolQ_K3a_1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
1,KhPolQ_K3a_1!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,3


In [20]:
coeff_df[['q-9_t-3', 'q9_t3', 'q-5_t-2', 'q5_t2']]

,q-9_t-3,q9_t3,q-5_t-2,q5_t2
0,1,0,1,0
1,0,1,0,1


In [13]:
## Add s-invariant
#pari.read("raw/Khovanov+Rasmussen/s-value_knots")
#s_list = [int(s) for s in pari('s_value_K13n')]
#s_list_double = []
#for s in s_list:
#    s_list_double.append(s)
#    s_list_double.append(-s)
coeff_df['s-value'] = pd.NA

In [14]:
signature_files = [
'03a',
'04a',
'05a',
'06a',
'07a',
'08a',
'08n',
'09a', 
'09n',
'10a',
'10n',
'11a',
'11n',
'12a',
'12n',
'13a',
'13n']

In [15]:
## Add signature
sign_list = []

for k in signature_files:
    signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-{}-Signature'.format(k), 
                              header=None, names = ['na','cross','mirror','signature'])
    signature_df.reset_index(inplace=True)
    sign_list += signature_df.signature.values.tolist()
    
coeff_df['signature'] = sign_list

In [16]:
coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [17]:
coeff_df

,knot_id,number_of_crossings,is_alternating,signature,s-value,q-39_t-13,q-39_t-12,q-39_t-11,q-39_t-10,q-39_t-9,...,q39_t4,q39_t5,q39_t6,q39_t7,q39_t8,q39_t9,q39_t10,q39_t11,q39_t12,q39_t13
0,KhPolQ_K3a_1,3,1,-2,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K3a_1!,3,1,2,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K4a_1,4,1,0,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KhPolQ_K4a_1!,4,1,0,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K5a_1,5,1,-2,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25925,KhPolQ_K13n_5108!,13,0,4,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25926,KhPolQ_K13n_5109,13,0,-2,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25927,KhPolQ_K13n_5109!,13,0,2,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25928,KhPolQ_K13n_5110,13,0,-2,<NA>,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
coeff_df.to_csv('parsed/Khovanov/KhPol_knots_data_UP_TO_13_MIRROR.csv', header=True, index=False)

# 12a

In [13]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_data")

Loading Khovanov polynomials for knots with up to 13 crossings ... done.


In [14]:
poly_vectors = ['KhPolQ_K12a']
min_max_list = [min_max_degree(pari(vec)) for vec in poly_vectors]

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])
size_q = max(abs(max_q), abs(min_q))

max_t = max([v['t']['max'] for v in min_max_list])
min_t = min([v['t']['min'] for v in min_max_list])
size_t = max(abs(max_t), abs(min_t))


print('q', min_q, max_q, size_q)
print('t', min_t, max_t, size_t)

q -25 33 33
t -10 12 12


In [15]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q, size_t, -size_t) 
        c_list.append(p_coeff)
        c_list.append(p_coeff.reshape((size_q*2 +1 , size_t*2+1))[::-1, ::-1].reshape(-1))

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

KhPolQ_K12a


|          | 0/? [00:00<?, ?it/s]

In [16]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q, size_t, -size_t) )
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

In [17]:
coeff_df

,knot_id,q-33_t-12,q-33_t-11,q-33_t-10,q-33_t-9,q-33_t-8,q-33_t-7,q-33_t-6,q-33_t-5,q-33_t-4,...,q33_t5,q33_t6,q33_t7,q33_t8,q33_t9,q33_t10,q33_t11,q33_t12,is_alternating,number_of_crossings
0,KhPolQ_K12a_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,12
1,KhPolQ_K12a_1!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,12
2,KhPolQ_K12a_2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,12
3,KhPolQ_K12a_2!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,12
4,KhPolQ_K12a_3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,KhPolQ_K12a_1286!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,12
2572,KhPolQ_K12a_1287,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,12
2573,KhPolQ_K12a_1287!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,12
2574,KhPolQ_K12a_1288,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,12


In [18]:
## Add s-invariant
pari.read("raw/Khovanov+Rasmussen/s-value_knots")
s_list = [int(s) for s in pari('s_value_K12a')]
s_list_double = []
for s in s_list:
    s_list_double.append(s)
    s_list_double.append(-s)
    
coeff_df['s-value'] = s_list_double

In [19]:
## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-12a-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
#signature_df = signature_df[~(signature_df.mirror.str[-1] == '!')]
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

In [20]:
coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [21]:
coeff_df

,knot_id,number_of_crossings,is_alternating,signature,s-value,q-33_t-12,q-33_t-11,q-33_t-10,q-33_t-9,q-33_t-8,...,q33_t3,q33_t4,q33_t5,q33_t6,q33_t7,q33_t8,q33_t9,q33_t10,q33_t11,q33_t12
0,KhPolQ_K12a_1,12,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K12a_1!,12,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K12a_2,12,1,-2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KhPolQ_K12a_2!,12,1,2,-2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K12a_3,12,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,KhPolQ_K12a_1286!,12,1,-4,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2572,KhPolQ_K12a_1287,12,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2573,KhPolQ_K12a_1287!,12,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2574,KhPolQ_K12a_1288,12,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
coeff_df.to_csv('parsed/Khovanov/KhPol_knots_data_12a_MIRROR.csv', header=True, index=False)

# 12n

In [11]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_data")

Loading Khovanov polynomials for knots with up to 13 crossings ... done.


In [12]:
poly_vectors = ['KhPolQ_K12n']
min_max_list = [min_max_degree(pari(vec)) for vec in poly_vectors]

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])
size_q = max(abs(max_q), abs(min_q))

max_t = max([v['t']['max'] for v in min_max_list])
min_t = min([v['t']['min'] for v in min_max_list])
size_t = max(abs(max_t), abs(min_t))


print('q', min_q, max_q, size_q)
print('t', min_t, max_t, size_t)

q -27 33 33
t -11 12 12


In [13]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q, size_t, -size_t) 
        c_list.append(p_coeff)
        c_list.append(p_coeff.reshape((size_q*2 +1 , size_t*2+1))[::-1, ::-1].reshape(-1))

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

KhPolQ_K12n


|          | 0/? [00:00<?, ?it/s]

In [14]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q, size_t, -size_t) )
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

## Add s-invariant
pari.read("raw/Khovanov+Rasmussen/s-value_knots")
s_list = [int(s) for s in pari('s_value_K12n')]
s_list_double = []
for s in s_list:
    s_list_double.append(s)
    s_list_double.append(-s)
    
coeff_df['s-value'] = s_list_double

## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-12a-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
#signature_df = signature_df[~(signature_df.mirror.str[-1] == '!')]
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [15]:
coeff_df

,knot_id,number_of_crossings,is_alternating,signature,s-value,q-33_t-12,q-33_t-11,q-33_t-10,q-33_t-9,q-33_t-8,...,q33_t3,q33_t4,q33_t5,q33_t6,q33_t7,q33_t8,q33_t9,q33_t10,q33_t11,q33_t12
0,KhPolQ_K12n_1,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K12n_1!,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K12n_2,12,0,-2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KhPolQ_K12n_2!,12,0,2,-2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K12n_3,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771,KhPolQ_K12n_886!,12,0,-2,-2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1772,KhPolQ_K12n_887,12,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1773,KhPolQ_K12n_887!,12,0,0,-4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1774,KhPolQ_K12n_888,12,0,-2,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
coeff_df.to_csv('parsed/Khovanov/KhPol_knots_data_12n_MIRROR.csv', header=True, index=False)

# 13a

In [7]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_data")

Loading Khovanov polynomials for knots with up to 13 crossings ... done.


In [8]:
poly_vectors = ['KhPolQ_K13a']
min_max_list = []

min_max_list.append(min_max_degree(pari('KhPolQ_K13a')[:1000]))
min_max_list.append(min_max_degree(pari('KhPolQ_K13a')[1000:2010]))
min_max_list.append(min_max_degree(pari('KhPolQ_K13a')[2000:3010]))
min_max_list.append(min_max_degree(pari('KhPolQ_K13a')[3000:4010]))
min_max_list.append(min_max_degree(pari('KhPolQ_K13a')[4000:]))

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])
size_q = max(abs(max_q), abs(min_q))

max_t = max([v['t']['max'] for v in min_max_list])
min_t = min([v['t']['min'] for v in min_max_list])
size_t = max(abs(max_t), abs(min_t))


print('q', min_q, max_q, size_q)
print('t', min_t, max_t, size_t)

q -31 39 39
t -11 13 13


In [9]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q, size_t, -size_t) 
        c_list.append(p_coeff)
        c_list.append(p_coeff.reshape((size_q*2 +1 , size_t*2+1))[::-1, ::-1].reshape(-1))

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

KhPolQ_K13a


|          | 0/? [00:00<?, ?it/s]

In [10]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q, size_t, -size_t) )
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

## Add s-invariant
pari.read("raw/Khovanov+Rasmussen/s-value_knots")
s_list = [int(s) for s in pari('s_value_K13a')]
s_list_double = []
for s in s_list:
    s_list_double.append(s)
    s_list_double.append(-s)
    
coeff_df['s-value'] = s_list_double

## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-13a-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
#signature_df = signature_df[~(signature_df.mirror.str[-1] == '!')]
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [11]:
coeff_df

,knot_id,number_of_crossings,is_alternating,signature,s-value,q-39_t-13,q-39_t-12,q-39_t-11,q-39_t-10,q-39_t-9,...,q39_t4,q39_t5,q39_t6,q39_t7,q39_t8,q39_t9,q39_t10,q39_t11,q39_t12,q39_t13
0,KhPolQ_K13a_1,13,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K13a_1!,13,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K13a_2,13,1,-2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KhPolQ_K13a_2!,13,1,2,-2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K13a_3,13,1,2,-2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9751,KhPolQ_K13a_4876!,13,1,6,-6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9752,KhPolQ_K13a_4877,13,1,-4,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9753,KhPolQ_K13a_4877!,13,1,4,-4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9754,KhPolQ_K13a_4878,13,1,-12,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [12]:
coeff_df.to_csv('parsed/Khovanov/KhPol_knots_data_13a_MIRROR.csv', header=True, index=False)

# 13n

In [ ]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_data")

In [ ]:
poly_vectors = ['KhPolQ_K13n']
min_max_list = []

min_max_list.append(min_max_degree(pari('KhPolQ_K13n')[:1000]))
min_max_list.append(min_max_degree(pari('KhPolQ_K13n')[1000:2010]))
min_max_list.append(min_max_degree(pari('KhPolQ_K13n')[2000:3010]))
min_max_list.append(min_max_degree(pari('KhPolQ_K13n')[3000:4010]))
min_max_list.append(min_max_degree(pari('KhPolQ_K13n')[4000:]))

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])
size_q = max(abs(max_q), abs(min_q))

max_t = max([v['t']['max'] for v in min_max_list])
min_t = min([v['t']['min'] for v in min_max_list])
size_t = max(abs(max_t), abs(min_t))


print('q', min_q, max_q, size_q)
print('t', min_t, max_t, size_t)

In [ ]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q, size_t, -size_t) 
        c_list.append(p_coeff)
        c_list.append(p_coeff.reshape((size_q*2 +1 , size_t*2+1))[::-1, ::-1].reshape(-1))

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

In [ ]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q, size_t, -size_t) )
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

## Add s-invariant
pari.read("raw/Khovanov+Rasmussen/s-value_knots")
s_list = [int(s) for s in pari('s_value_K13n')]
s_list_double = []
for s in s_list:
    s_list_double.append(s)
    s_list_double.append(-s)
    
coeff_df['s-value'] = s_list_double

## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-13n-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
#signature_df = signature_df[~(signature_df.mirror.str[-1] == '!')]
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [ ]:
coeff_df

In [ ]:
coeff_df.to_csv('parsed/Khovanov/KhPol_knots_data_13n_MIRROR.csv', header=True, index=False)

# 14n

In [ ]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_14n_data")

In [ ]:
poly_vectors = ['KhPolQ_K14n']
min_max_list = []

for i in tqdm(range(0, len(pari('KhPolQ_K14n')), 1000)):
    #print(i, i+1000)
    min_max_list.append(min_max_degree(pari('KhPolQ_K14n')[i:i+1000]))

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])
size_q = max(abs(max_q), abs(min_q))

max_t = max([v['t']['max'] for v in min_max_list])
min_t = min([v['t']['min'] for v in min_max_list])
size_t = max(abs(max_t), abs(min_t))


print('q', min_q, max_q, size_q)
print('t', min_t, max_t, size_t)

In [ ]:
c_list = []
knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q, size_t, -size_t) 
        c_list.append(p_coeff)
        c_list.append(p_coeff.reshape((size_q*2 +1 , size_t*2+1))[::-1, ::-1].reshape(-1))

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')

In [ ]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q, size_t, -size_t) )
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

## Add s-invariant
pari.read("raw/Khovanov+Rasmussen/s-value_14n")
s_list = [int(s) for s in pari('s_value_K14n')]
s_list_double = []
for s in s_list:
    s_list_double.append(s)
    s_list_double.append(-s)
    
coeff_df['s-value'] = s_list_double

## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-14n-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
#signature_df = signature_df[~(signature_df.mirror.str[-1] == '!')]
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [ ]:
coeff_df

In [ ]:
coeff_df.to_csv('parsed/Khovanov/KhPol_knots_data_14n_MIRROR.csv', header=True, index=False)

# 15n

In [ ]:
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_15n_data")

In [ ]:
poly_vectors = ['KhPolQ_K15n']
min_max_list = []

print(len(pari('KhPolQ_K15n')))
for i in tqdm(range(0, len(pari('KhPolQ_K15n')), 1000)):
    #print(i, i+1000)
    min_max_list.append(min_max_degree(pari('KhPolQ_K15n')[i:i+1000]))

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])
size_q = max(abs(max_q), abs(min_q))

max_t = max([v['t']['max'] for v in min_max_list])
min_t = min([v['t']['min'] for v in min_max_list])
size_t = max(abs(max_t), abs(min_t))


print('q', min_q, max_q, size_q)
print('t', min_t, max_t, size_t)

In [ ]:
c_list = []
c_list_or = []
c_list_mir = []

knot_id_list = []
knot_id_list_or = []
knot_id_list_mir = []


for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):
        p_coeff = create_coeff_vector(p, size_q, -size_q, size_t, -size_t) 
        c_list.append(p_coeff)
        c_list.append(p_coeff.reshape((size_q*2 +1 , size_t*2+1))[::-1, ::-1].reshape(-1))

        c_list_or.append(p_coeff)
        c_list_mir.append(p_coeff.reshape((size_q*2 +1 , size_t*2+1))[::-1, ::-1].reshape(-1))

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')
        
        knot_id_list_or.append(poly_list+'_'+str(i+1))
        knot_id_list_mir.append(poly_list+'_'+str(i+1)+'!')

In [ ]:
import pickle

In [ ]:
fileObj = open('c_list.obj', 'wb')
pickle.dump(c_list, fileObj)
fileObj.close()

In [ ]:
fileObj = open('c_list_or.obj', 'wb')
pickle.dump(c_list_or, fileObj)
fileObj.close()

In [ ]:
fileObj = open('c_list_mir.obj', 'wb')
pickle.dump(c_list_mir, fileObj)
fileObj.close()

In [ ]:
import pickle   

fileObj = open('c_list.obj', 'rb')
c_list = pickle.load(fileObj)
fileObj.close()

In [ ]:
#np.savetxt("foo.csv", exampleObj, delimiter=",", fmt='%d')

In [ ]:
poly_vectors = ['KhPolQ_K15n']

knot_id_list = []

for poly_list in poly_vectors:
    print(poly_list)
    
    for i, p in tqdm(enumerate(pari(poly_list))):

        knot_id_list.append(poly_list+'_'+str(i+1))
        knot_id_list.append(poly_list+'_'+str(i+1)+'!')
        
size_q = 41
size_t = 15

In [ ]:
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q, size_t, -size_t) )
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

## Add s-invariant
pari.read("raw/Khovanov+Rasmussen/s-value_15n")
s_list = [int(s) for s in pari('s_value_K15n')]
s_list_double = []
for s in s_list:
    s_list_double.append(s)
    s_list_double.append(-s)
    
coeff_df['s-value'] = s_list_double

## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-15n-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
#signature_df = signature_df[~(signature_df.mirror.str[-1] == '!')]
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature', 's-value'] 
                     + coeff_df.columns.to_list()[1:-4]]

In [ ]:
coeff_df

In [ ]:
coeff_df.to_csv('parsed/Khovanov/KhPol_knots_data_15n_MIRROR.csv', header=True, index=False)

# 16n

In [7]:
%%time
pari.read("raw/Khovanov+Rasmussen/KhPol_knots_16n_data")

Loading Khovanov polynomials for non-alternating knots with 16 crossings ... done.
CPU times: user 23min 13s, sys: 14.4 s, total: 23min 28s
Wall time: 23min 55s


In [ ]:
poly_vectors = ['KhPolQ_K16n']
min_max_list = []

print(len(pari('KhPolQ_K16n')))

for poly in tqdm(pari('KhPolQ_K16n')):
      min_max_list.append(min_max_degree_single(poly))

max_q = max([v['q']['max'] for v in min_max_list])
min_q = min([v['q']['min'] for v in min_max_list])
size_q = max(abs(max_q), abs(min_q))

max_t = max([v['t']['max'] for v in min_max_list])
min_t = min([v['t']['min'] for v in min_max_list])
size_t = max(abs(max_t), abs(min_t))


print('q', min_q, max_q, size_q)
print('t', min_t, max_t, size_t)

# q -41 45 45
# t -15 16 16

In [4]:
size_q = 45
size_t = 16

In [11]:
c_list = []
knot_id_list = []

for i, p in tqdm(enumerate(pari('KhPolQ_K16n'))):
    p_coeff = create_coeff_vector(p, size_q, -size_q, size_t, -size_t) 
    c_list.append(p_coeff)
    c_list.append(p_coeff.reshape((size_q*2 +1 , size_t*2+1))[::-1, ::-1].reshape(-1))

    knot_id_list.append('KhPolQ_K16n'+'_'+str(i+1))
    knot_id_list.append('KhPolQ_K16n'+'_'+str(i+1)+'!')

|          | 0/? [00:00<?, ?it/s]

In [5]:
with open('c_list_16n.npy', 'wb') as f:
    np.save(f, c_list)
    
with open('knot_id_list_16n.npy', 'wb') as k:
    np.save(k, knot_id_list)

NameError: name 'c_list' is not defined

In [25]:
%%time
c_array = np.array(c_list)

CPU times: user 30.8 s, sys: 1min 24s, total: 1min 55s
Wall time: 4min 10s


In [29]:
%%time
c_array.min()

CPU times: user 16.8 s, sys: 58.8 s, total: 1min 15s
Wall time: 2min 25s


0

In [31]:
%%time
with open('c_array_16n.npy', 'wb') as y:
    np.save(y, c_array)

CPU times: user 8.78 s, sys: 5min 22s, total: 5min 30s
Wall time: 9min 30s


In [6]:
%%time
with open('c_array_16n.npy', 'rb') as f:
    c_list = np.load(f)
    
with open('knot_id_list_16n.npy', 'rb') as k:
    knot_id_list = np.load(k)

CPU times: user 9.63 ms, sys: 1min 16s, total: 1min 16s
Wall time: 2min 50s


In [7]:
%%time
coeff_df = pd.DataFrame(data=c_list, 
                        columns=create_colnames(size_q, -size_q, size_t, -size_t) )
coeff_df['knot_id'] = knot_id_list

# put knot_id in the first place
#coeff_df = coeff_df[['knot_id'] + coeff_df.columns.to_list()[:-1]]

coeff_df['is_alternating'] = [int(kid.split('_')[1][-1] == 'a') for kid in coeff_df.knot_id]
coeff_df['number_of_crossings'] = [int(kid.split('_')[1][1:-1]) for kid in coeff_df.knot_id]

## Add s-invariant
# pari.read("raw/Khovanov+Rasmussen/s-value_15n")
# s_list = [int(s) for s in pari('s_value_K15n')]
# s_list_double = []
# for s in s_list:
#     s_list_double.append(s)
#     s_list_double.append(-s)
    
#coeff_df['s-value'] = s_list_double
#coeff_df['s-value'] = np.na

## Add signature
signature_df = pd.read_csv('raw/Khovanov+Rasmussen/Signature/knots-16n-Signature', 
                          header=None, names = ['na','cross','mirror','signature'])
#signature_df = signature_df[~(signature_df.mirror.str[-1] == '!')]
signature_df.reset_index(inplace=True)
coeff_df['signature'] = signature_df.signature

CPU times: user 7.94 s, sys: 1.09 s, total: 9.03 s
Wall time: 9.3 s


In [ ]:
coeff_df = coeff_df[['knot_id', 'number_of_crossings', 'is_alternating', 'signature'] 
                     + coeff_df.columns.to_list()[1:-3]]

In [8]:
coeff_df.head()

,q-45_t-16,q-45_t-15,q-45_t-14,q-45_t-13,q-45_t-12,q-45_t-11,q-45_t-10,q-45_t-9,q-45_t-8,q-45_t-7,...,q45_t11,q45_t12,q45_t13,q45_t14,q45_t15,q45_t16,knot_id,is_alternating,number_of_crossings,signature
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,KhPolQ_K16n_1,0,16,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,KhPolQ_K16n_1!,0,16,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,KhPolQ_K16n_2,0,16,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,KhPolQ_K16n_2!,0,16,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,KhPolQ_K16n_3,0,16,2


In [9]:
coeff_df.to_csv('parsed/Khovanov/KhPol_knots_data_16n_MIRROR.csv', header=True, index=False)

# merge files

In [1]:
import pandas as pd

In [2]:
%%time
khov_13 = pd.read_csv('parsed/Khovanov/KhPol_knots_data_UP_TO_13_MIRROR.csv')
khov_14n = pd.read_csv('parsed/Khovanov/KhPol_knots_data_14n_MIRROR.csv')
#khov_15n = pd.read_csv('parsed/Khovanov/KhPol_knots_data_15n_MIRROR.csv')
#khov_16n = pd.read_csv('parsed/Khovanov/KhPol_knots_data_16n_MIRROR.csv')

CPU times: user 39.9 s, sys: 5.41 s, total: 45.3 s
Wall time: 48.4 s


In [3]:
khov_13n = khov_13[khov_13.is_alternating == 0].copy()

In [4]:
print(khov_13n.shape)
print(khov_14n.shape)
#print(khov_15n.shape)
#print(khov_16n.shape)

(12472, 2138)
(54872, 2296)


In [7]:
missing_cols = [c for c in khov_14n.columns if c not in khov_13n.columns]
missing_cols

['q-39_t-14',
 'q-39_t14',
 'q-38_t-14',
 'q-38_t14',
 'q-37_t-14',
 'q-37_t14',
 'q-36_t-14',
 'q-36_t14',
 'q-35_t-14',
 'q-35_t14',
 'q-34_t-14',
 'q-34_t14',
 'q-33_t-14',
 'q-33_t14',
 'q-32_t-14',
 'q-32_t14',
 'q-31_t-14',
 'q-31_t14',
 'q-30_t-14',
 'q-30_t14',
 'q-29_t-14',
 'q-29_t14',
 'q-28_t-14',
 'q-28_t14',
 'q-27_t-14',
 'q-27_t14',
 'q-26_t-14',
 'q-26_t14',
 'q-25_t-14',
 'q-25_t14',
 'q-24_t-14',
 'q-24_t14',
 'q-23_t-14',
 'q-23_t14',
 'q-22_t-14',
 'q-22_t14',
 'q-21_t-14',
 'q-21_t14',
 'q-20_t-14',
 'q-20_t14',
 'q-19_t-14',
 'q-19_t14',
 'q-18_t-14',
 'q-18_t14',
 'q-17_t-14',
 'q-17_t14',
 'q-16_t-14',
 'q-16_t14',
 'q-15_t-14',
 'q-15_t14',
 'q-14_t-14',
 'q-14_t14',
 'q-13_t-14',
 'q-13_t14',
 'q-12_t-14',
 'q-12_t14',
 'q-11_t-14',
 'q-11_t14',
 'q-10_t-14',
 'q-10_t14',
 'q-9_t-14',
 'q-9_t14',
 'q-8_t-14',
 'q-8_t14',
 'q-7_t-14',
 'q-7_t14',
 'q-6_t-14',
 'q-6_t14',
 'q-5_t-14',
 'q-5_t14',
 'q-4_t-14',
 'q-4_t14',
 'q-3_t-14',
 'q-3_t14',
 'q-2_t-14',
 '

In [9]:
khov_13n[missing_cols] = 0

In [10]:
print(khov_13n.shape)
print(khov_14n.shape)

(12472, 2296)
(54872, 2296)


In [11]:
%%time
khov_upto_14n = pd.concat([khov_13n, khov_14n], ignore_index=True)

CPU times: user 1.78 s, sys: 2.3 s, total: 4.08 s
Wall time: 4.44 s


In [18]:
khov_upto_14n.to_csv('parsed/Khovanov_upto_14n_MIRROR.csv', header=True, index=False)

In [16]:
khov_upto_14n.drop(columns=['s-value'], inplace=True)

In [17]:
khov_upto_14n.head()

,knot_id,number_of_crossings,is_alternating,signature,q-39_t-13,q-39_t-12,q-39_t-11,q-39_t-10,q-39_t-9,q-39_t-8,...,q35_t-14,q35_t14,q36_t-14,q36_t14,q37_t-14,q37_t14,q38_t-14,q38_t14,q39_t-14,q39_t14
0,KhPolQ_K8n_1,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K8n_1!,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K8n_2,8,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KhPolQ_K8n_2!,8,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K8n_3,8,0,-6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 15

In [20]:
del khov_13
del khov_13n
del khov_14n

khov_15n = pd.read_csv('parsed/Khovanov/KhPol_knots_data_15n_MIRROR.csv')

In [21]:
missing_cols = [c for c in khov_15n.columns if c not in khov_upto_14n.columns]
khov_upto_14n[missing_cols] = 0

In [22]:
%%time
khov_upto_15n = pd.concat([khov_upto_14n, khov_15n], ignore_index=True)

CPU times: user 16.9 s, sys: 28.8 s, total: 45.7 s
Wall time: 53 s


In [23]:
%%time
khov_upto_15n.to_csv('parsed/Khovanov_upto_15n_MIRROR.csv', header=True, index=False)

CPU times: user 5min 28s, sys: 12.9 s, total: 5min 41s
Wall time: 5min 46s


In [24]:
print(khov_upto_14n.shape)
print(khov_15n.shape)
print(khov_upto_15n.shape)

(67344, 2578)
(336060, 2578)
(403404, 2578)


In [25]:
khov_upto_15n.head()

,knot_id,number_of_crossings,is_alternating,signature,q-39_t-13,q-39_t-12,q-39_t-11,q-39_t-10,q-39_t-9,q-39_t-8,...,q41_t6,q41_t7,q41_t8,q41_t9,q41_t10,q41_t11,q41_t12,q41_t13,q41_t14,q41_t15
0,KhPolQ_K8n_1,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K8n_1!,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K8n_2,8,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KhPolQ_K8n_2!,8,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K8n_3,8,0,-6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 16

In [1]:
import pandas as pd

In [2]:
%%time
khov_upto_15n = pd.read_csv('parsed/Khovanov_upto_15n_MIRROR.csv')
khov_16n = pd.read_csv('parsed/Khovanov/KhPol_knots_data_16n_MIRROR.csv')

CPU times: user 28min 6s, sys: 6min 36s, total: 34min 42s
Wall time: 37min 59s


In [3]:
missing_cols = [c for c in khov_16n.columns if c not in khov_upto_15n.columns]
khov_upto_15n[missing_cols] = 0

In [4]:
%%time
khov_upto_16n = pd.concat([khov_upto_15n, khov_16n], ignore_index=True)

CPU times: user 2min 5s, sys: 3min 57s, total: 6min 3s
Wall time: 6min 51s


In [5]:
print(khov_upto_16n.shape)
print(khov_16n.shape)
print(khov_upto_16n.shape)

(2421216, 3008)
(2017812, 3007)
(2421216, 3008)


In [6]:
del khov_upto_15n

In [7]:
khov_upto_16n.head()

,knot_id,number_of_crossings,is_alternating,signature,q-39_t-13,q-39_t-12,q-39_t-11,q-39_t-10,q-39_t-9,q-39_t-8,...,q45_t7,q45_t8,q45_t9,q45_t10,q45_t11,q45_t12,q45_t13,q45_t14,q45_t15,q45_t16
0,KhPolQ_K8n_1,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,KhPolQ_K8n_1!,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,KhPolQ_K8n_2,8,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,KhPolQ_K8n_2!,8,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,KhPolQ_K8n_3,8,0,-6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
%%time
khov_upto_16n.to_csv('parsed/Khovanov_upto_16n_MIRROR.csv', header=True, index=False)

CPU times: user 41min 45s, sys: 2min 15s, total: 44min
Wall time: 45min 1s
